<a href="https://colab.research.google.com/github/Vampaxx/Pyspark_basics/blob/main/withcolumn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a23537a0426aaa2e9922e7cf560b5efd35d6d3088c773138ca4b594e17f153c8
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
csv_file = ('/content/drive/MyDrive/Pyspark/datas/Clean_Dataset.csv')

Pyspark withcolumn() is a trasnformation function of dataframe which is used to change the value, convert the datatype of an existing column, create new column and many more

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit,when

# create session
spark = SparkSession.builder.appName('withcolumn').getOrCreate()

In [33]:
data = [(1,'Shah',50000),
        (2,'Micheal',75000),
        (3,'Mia',10000),
        (4,'Daniel',2500),
        (5,'Johnson',6000)]
columns = ['Id','Emp_name','Salary']
df = spark.createDataFrame(data,schema = columns)
df.show()
df.count()
df.printSchema()

+---+--------+------+
| Id|Emp_name|Salary|
+---+--------+------+
|  1|    Shah| 50000|
|  2| Micheal| 75000|
|  3|     Mia| 10000|
|  4|  Daniel|  2500|
|  5| Johnson|  6000|
+---+--------+------+

root
 |-- Id: long (nullable = true)
 |-- Emp_name: string (nullable = true)
 |-- Salary: long (nullable = true)



In [25]:
df.Id+2

Column<'(Id + 2)'>

In [34]:
df.show()
df.withColumn(colName='Salary',col=col('Salary').cast('Integer')).printSchema()

# change existing column value
df.withColumn('Salary',col('Salary')*2).show()


df.withColumn('Salary+Increament',col('Salary')+col('Salary')*0.1).show()

df.withColumn('Department',lit('Production')).show()

+---+--------+------+
| Id|Emp_name|Salary|
+---+--------+------+
|  1|    Shah| 50000|
|  2| Micheal| 75000|
|  3|     Mia| 10000|
|  4|  Daniel|  2500|
|  5| Johnson|  6000|
+---+--------+------+

root
 |-- Id: long (nullable = true)
 |-- Emp_name: string (nullable = true)
 |-- Salary: integer (nullable = true)

+---+--------+------+
| Id|Emp_name|Salary|
+---+--------+------+
|  1|    Shah|100000|
|  2| Micheal|150000|
|  3|     Mia| 20000|
|  4|  Daniel|  5000|
|  5| Johnson| 12000|
+---+--------+------+

+---+--------+------+-----------------+
| Id|Emp_name|Salary|Salary+Increament|
+---+--------+------+-----------------+
|  1|    Shah| 50000|          55000.0|
|  2| Micheal| 75000|          82500.0|
|  3|     Mia| 10000|          11000.0|
|  4|  Daniel|  2500|           2750.0|
|  5| Johnson|  6000|           6600.0|
+---+--------+------+-----------------+

+---+--------+------+----------+
| Id|Emp_name|Salary|Department|
+---+--------+------+----------+
|  1|    Shah| 50000|Prod

In [35]:
df1 = df.withColumn('Department',
              when(col('Emp_name') == 'Mia',lit('Production'))
              .when(col('Emp_name') == 'Daniel', lit('Data analyst'))
              .when(col('Emp_name') == 'Johnson',lit('Data engineer'))
              .otherwise(lit('Associate'))
              )

In [37]:
df1.show()

+---+--------+------+-------------+
| Id|Emp_name|Salary|   Department|
+---+--------+------+-------------+
|  1|    Shah| 50000|    Associate|
|  2| Micheal| 75000|    Associate|
|  3|     Mia| 10000|   Production|
|  4|  Daniel|  2500| Data analyst|
|  5| Johnson|  6000|Data engineer|
+---+--------+------+-------------+



In [46]:
df2 = df1.withColumn('working_hours',
                      when(col('Department') == 'Associate',lit(10))
                      .when(col('Department') == 'Production',lit(7))
                      .when(col('Department') == 'Data analyst',lit(6))
                      .when(col('Department') == 'Data engineer',lit(8))
                      .otherwise(lit(None))
                      )
df2 = df2.withColumn(colName='Id',col=col('Id').cast('Integer'))

In [47]:
df2.show()

+---+--------+------+-------------+-------------+
| Id|Emp_name|Salary|   Department|working_hours|
+---+--------+------+-------------+-------------+
|  1|    Shah| 50000|    Associate|           10|
|  2| Micheal| 75000|    Associate|           10|
|  3|     Mia| 10000|   Production|            7|
|  4|  Daniel|  2500| Data analyst|            6|
|  5| Johnson|  6000|Data engineer|            8|
+---+--------+------+-------------+-------------+



In [48]:
df2.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Emp_name: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Department: string (nullable = false)
 |-- working_hours: integer (nullable = true)



In [58]:
from pyspark.sql.functions import concat, lit

df3 = df2.withColumn('Emp_department', concat(df2['Emp_name'], lit('--'), df2['Department']))
df3.show(truncate=False)
df3.printSchema()

+---+--------+------+-------------+-------------+----------------------+
|Id |Emp_name|Salary|Department   |working_hours|Emp_department        |
+---+--------+------+-------------+-------------+----------------------+
|1  |Shah    |50000 |Associate    |10           |Shah--Associate       |
|2  |Micheal |75000 |Associate    |10           |Micheal--Associate    |
|3  |Mia     |10000 |Production   |7            |Mia--Production       |
|4  |Daniel  |2500  |Data analyst |6            |Daniel--Data analyst  |
|5  |Johnson |6000  |Data engineer|8            |Johnson--Data engineer|
+---+--------+------+-------------+-------------+----------------------+

root
 |-- Id: integer (nullable = true)
 |-- Emp_name: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Department: string (nullable = false)
 |-- working_hours: integer (nullable = true)
 |-- Emp_department: string (nullable = true)

